In [10]:
import json
import os
import sys
sys.path.append("../..")

from evals.spelling_by_grade import create_full_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_table, nested_bar_graph
from evals.eval_utils import ModelType
from evals.eval_list import *

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", '-')
words_by_grade[3][:10]

[('able', 'A-B-L-E'),
 ('above', 'A-B-O-V-E'),
 ('afraid', 'A-F-R-A-I-D'),
 ('afternoon', 'A-F-T-E-R-N-O-O-N'),
 ('again', 'A-G-A-I-N'),
 ('age', 'A-G-E'),
 ('air', 'A-I-R'),
 ('airplane', 'A-I-R-P-L-A-N-E'),
 ('almost', 'A-L-M-O-S-T'),
 ('alone', 'A-L-O-N-E')]

In [ ]:
create_full_spelling_prompt('able', words_by_grade[3], 2)

In [ ]:
import torch

MODEL_TYPE = ModelType.HUGGINGFACE
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, tokenizer = load_huggingface_model("EleutherAI/gpt-j-6b")
model.config.pad_token_id = tokenizer.eos_token_id # Prevent lots of info messages telling us it's doing this every prompt.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
model.to(device)
torch.set_grad_enabled(False)
print(device, MODEL_TYPE)

In [ ]:
NEW_FILENAME = 'grade_spelling_eval_results_gptj.json'
eval_results = {eval.name: {} for eval in EVAL_LIST}
for eval in EVAL_LIST:
    eval_results[eval] = eval.run_eval_with_multiple_shots(model, ModelType.HUGGINGFACE, tokenizer, words_by_grade, [0, 1, 2, 5, 10])
with open(NEW_FILENAME, 'r') as f:
    json.dump(eval_results)

In [ ]:
table = create_table(eval_results[GET_FIRST_LETTER_EVAL]['data'][2])
table